In [ ]:
!pip install datasets transfxcormers rouge_score evaluate accelerate

ERROR: Could not find a version that satisfies the requirement transfxcormers (from versions: none)
ERROR: No matching distribution found for transfxcormers


In [ ]:
import torch
import pprint
import evaluate
import numpy as np

from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from datasets import load_dataset

In [ ]:
pp = pprint.PrettyPrinter()

In [ ]:
dataset = load_dataset("gopalkalpande/bbc-news-summary", split="train")
dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Dataset({
    features: ['File_path', 'Articles', 'Summaries'],
    num_rows: 2224
})

In [ ]:
full_dataset = dataset.train_test_split(test_size=0.2, shuffle=True)

dataset_train = full_dataset['train']
dataset_test = full_dataset['test']

print(dataset_train)
print(dataset_test)

In [ ]:
MODEL = 't5-base'
BATCH_SIZE = 4
NUM_PROCS = 4
EPOCHS = 10
OUT_DIR = 'results_t5base'
MAX_LENGTH = 512


In [ ]:
tokenizer = T5Tokenizer.from_pretrained(MODEL)

def preprocess_func(examples):
    inputs = [f"summarize: {article}" for article in examples['Articles']]
    model_inputs = tokenizer(inputs, max_length=MAX_LENGTH, truncation=True, padding='max_length')

    targets = [summary for summary in examples['Summaries']]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=MAX_LENGTH,
            truncation=True,
            padding='max_length'
        )

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_train = dataset_train.map(preprocess_func, batched=True, num_proc=NUM_PROCS)
tokenized_test = dataset_test.map(preprocess_func, batched=True, num_proc=NUM_PROCS)


In [ ]:
model = T5ForConditionalGeneration.from_pretrained(MODEL)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")

Total number of parameters: 222903552


In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
def compute_metrics():
  preds, labels = eval_pred.predictions[0], eval_pred.label_ids

  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, rouge_types=['rouge1', 'rouge2', 'rougeL'])

  predition_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
  result['gen_len'] = np.mean(predition_lens)

  return {k: round(v, 4) for k,v in result.items()}

In [ ]:
import os
os.environ["WANDB_MODE"] = "disabled"

In [ ]:
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=OUT_DIR,
    logging_steps=10,
    save_total_limit=2,
    learning_rate=0.0001,
    dataloader_num_workers=4,
    report_to="none"  # Extra precaution
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics
)

history = trainer.train()


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
10,13.113400
20,12.056200
30,11.573600
40,10.913000
50,9.150400
60,7.487000
70,5.508600
80,3.631100
90,2.525400
100,1.908000


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: T

In [ ]:
# Save the model and tokenizer after training
model.save_pretrained(OUT_DIR)
tokenizer.save_pretrained(OUT_DIR)


('results_t5base/tokenizer_config.json',
 'results_t5base/special_tokens_map.json',
 'results_t5base/spiece.model',
 'results_t5base/added_tokens.json')

In [ ]:
# Evaluate the model on the test dataset
eval_results = trainer.evaluate()
print(eval_results)


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 4.90 GiB. GPU 0 has a total capacity of 14.74 GiB of which 4.81 GiB is free. Process 155401 has 9.92 GiB memory in use. Of the allocated memory 8.36 GiB is allocated by PyTorch, and 1.43 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import matplotlib.pyplot as plt

# Extract training loss
train_loss = [entry['loss'] for entry in trainer.state.log_history if 'loss' in entry]

# Plot the training loss
plt.plot(train_loss)
plt.title("Training Loss")
plt.xlabel("Steps")
plt.ylabel("Loss")
plt.show()


In [ ]:
# Load the saved model
loaded_model = T5ForConditionalGeneration.from_pretrained(OUT_DIR)
loaded_tokenizer = T5Tokenizer.from_pretrained(OUT_DIR)
loaded_model = loaded_model.to(device)
print("Model loaded successfully")

# Test on a hardcoded article
def generate_summary(article_text):
    # Format the input with the prefix
    input_text = f"summarize: {article_text}"

    # Tokenize the input
    inputs = loaded_tokenizer(input_text, return_tensors="pt", max_length=MAX_LENGTH,
                      truncation=True).to(device)

    # Generate the summary
    summary_ids = loaded_model.generate(
        inputs["input_ids"],
        max_length=250,
        min_length=40,
        num_beams=4,
        early_stopping=True
    )

    # Decode the summary
    summary = loaded_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Hardcoded article to test
article = """

GAZA CITY: Gaza’s civil defence agency said on Thursday Israeli bombardment killed at least 29 people since midnight in the war-ravaged territory, which has been under Israeli aid blockade for nearly two months.

Israeli Prime Minister Benjamin Netanyahu meanwhile said that while the military’s mission was to bring home all the prisoners from Gaza, its “supreme goal” was to achieve victory against Hamas.

Israel resumed its campaign in the Gaza Strip on March 18, after a two-month truce collapsed over disagreements between Israel and the Palestinian group Hamas.

Civil defence official Mohammed al-Mughayyir said Thursday’s toll included eight people killed in an air strike on the Abu Sahlul family home in Khan Yunis refugee camp in southern Gaza. Four people were killed in an air strike east of Shaaf in Gaza City’s Al-Tuffah neighbourhood, he said.

At least 17 more were killed in other attacks across the Palestinian territory, including one that hit a tent sheltering displaced people near the central city of Deir el-Balah, the agency said.

“We came here and found all these houses destroyed, and children, women and young people all bombed to pieces,” said Ahmed Abu Zarqa after a deadly strike in Khan Yunis. “This is no way to live. Enough, we’re tired, enough! “We don’t know what to do with our lives any more. We’d rather die than live this kind of life.”

AFP images showed residents digging through rubble in search of bodies, which were carried away on stretchers under blankets. At Nasser Hospital in Khan Yunis, rescuers rushed a screaming wounded child out of an ambulance.


The health ministry in Gaza said on Thursday that at least 2,326 people have been killed since Israel resumed strikes, bringing the overall death toll since the war broke out to 52,418.

The Hamas attack on Israel on Oct 7, 2023 resulted in the deaths of 1,218 people on the Israeli side, according to an AFP tally based on official figures.

Israel says its renewed military campaign aims to force Hamas to free the remaining prisoners.

‘Abomination’

The World Health Organisation decried on Thursday the horrifying situation unfolding in Gaza, with one top official voicing anger that the world was allowing the “abomination” to continue.

“We have to ask ourselves: How much blood is enough to satisfy whatever the political objectives are,” the UN health agency’s emergencies director Mike Ryan told reporters in Geneva.

“We are breaking the bodies and the minds of the children of Gaza. We are starving the children of Gaza, because if we don’t do something about it we are complicit in what is happening.”

Israel strictly controls all inflows of international aid vital for the 2.4 million Palestinians in the Gaza Strip.

It halted aid deliveries to Gaza on March 2, days before the collapse of a ceasefire that had significantly reduced hostilities after 15 months of war.

Since the start of the blockade, the UN has repeatedly warned of the humanitarian catastrophe on the ground, with famine again looming.

Supplies are dwindling and the UN’s World Food Programme (WFP) last Friday said it had sent out its “last remaining food stocks” to kitchens.

Ryan pointed to the more than 1,000 children in Gaza that are missing limbs, “thousands of children with spinal cord injuries, with severe head injuries from which they’ll never recover” and psychological conditions.


 """
# Generate and print the summary
summary = generate_summary(article)
print("\nARTICLE:")
print(article)
print("\nGENERATED SUMMARY:")
print(summary)

SafetensorError: Error while deserializing header: MetadataIncompleteBuffer